# CODE - 1

In [ ]:
class LinearSolve:
    def __init__(self,matrix,vector) -> None:
        self.matrix = matrix
        self.vector = vector
        self.res = np.array([])
        self.err = 1e-7
    def get_upper_permute(self,matrix):
        n = len(matrix)
        upper = copy.deepcopy(matrix)
        permute = np.eye(n)
        for i in range(n):
            if(upper[i][i] == 0):
                j = i+1
                while(j < n and upper[j][i] == 0):
                    j += 1
                if(j == n):
                    continue
                else:
                    permute[[i,j]] = permute[[j,i]]
                    upper = np.dot(permute,upper)
            for j in range(i+1,n):
                m = upper[j][i]/upper[i][i]
                upper[j][i] = 0
                for k in range(i+1,n+1):
                    upper[j][k] = upper[j][k] - m*upper[i][k]
        return upper,permute
    
    def get_inv(self,matrix):
        matinv = np.linalg.inv(matrix)
        return matinv

    def get_lower(self,matrix,upper,permute):
        pa = np.dot(permute,matrix)
        upp_inv = self.get_inv(upper)
        return np.dot(pa,upp_inv)

    def gauss(self,A_n,b):
        n = len(A_n)
        A = copy.deepcopy(A_n)
        #st = deque()
        for i in range(n):
            A[i].append(b[i])
        
        A,_ = self.get_upper_permute(A)
        x = [0]*n
        for i in range(n-1,-1,-1):
            x[i] = A[i][n]
            k = 0
            for j in range(n-1,i,-1):
                x_j = x[j]*A[i][j]
                if(abs(x_j) < self.err):
                    continue
                x[i] -= x[j]*A[i][j]
            x[i] /= A[i][i]
            if(abs(x[i]) < self.err):
                x[i] = 0
        self.res = x
        return np.array(x)
    
    def vector_norm(self,x1,x2):
        return np.linalg.norm(np.array(x1) - np.array(x2))
    
    def jacobi(self,A,b,guess):
        C,permute = self.diagonal_dominance(A)
        if C == False:
            return None,None,None
        n = len(A)
        iter = []
        ite = 1
        norm = []
        x = copy.deepcopy(guess)
        mat = [0]*n
        norm_val = 0
        new_x = copy.deepcopy(x)
        while True:
            for i in range(n):
                sum_val = 0
                for j in range(n):
                    if i != j :
                        sum_val -= C[i][j]*x[j]
                new_x[i] = (b[i] + sum_val)/C[i][i]
            
            norm_val = self.vector_norm(new_x,x)
            # print(new_x)
            # print(x)
            if(norm_val < self.err):
                break
            iter.append(ite)
            ite += 1
            norm.append(norm_val)
            x = copy.deepcopy(new_x)
        
        new_x = copy.deepcopy(x)
        for i in range(n):
            new_x[i] = x[permute[i]]
        x = new_x
        return np.array(x),iter,norm
    
    def diagonal_dominance(self,A):
        flag = True
        n = len(A)
        column_idx = [0]*n
        uset = set()
        for i in range(n):
            column_idx[i] = A[i].index(max(A[i],key=abs))
            #print(column_idx[i])
            if column_idx[i] in uset:
                print('Matrix is not Diagonally Dominant and hence result cannot be computed')
                return False,column_idx
            val = 0
            for j in range(n):
                if j != column_idx[i]:
                    val += abs(A[i][j])
            if val >= abs(A[i][column_idx[i]]):
                print('Matrix is not Diagonally Dominant and hence result cannot be computed')
                return False,column_idx
            uset.add(column_idx[i])
        C = copy.deepcopy(A)
        for i in range(n):
            if i != column_idx[i]:
                flag = False
            C[column_idx[i]] = A[i]
        if not flag:
            print('Matrix is not Diagonally Dominant but permutation of rows can make it dominant')
        else:
            print('Matrix is Diagonally Dominant')
        return C,column_idx

    def seidal(self,A,b,guess):
        C,permute = self.diagonal_dominance(A)
        if C == False:
            return None,None,None
        n = len(A)
        x = copy.deepcopy(guess)
        iter = []
        norm = []
        ite = 1
        mat = [0]*n
        norm_val = 0
        old_x = copy.deepcopy(x)
        while True:
            for i in range(n):
                sum_val = 0
                for j in range(n):
                    if i != j :
                        sum_val -= C[i][j]*x[j]
                x[i] = (b[i] + sum_val)/C[i][i]
            #print(x)
            norm_val = self.vector_norm(x,old_x)
            if(norm_val < self.err):
                break
            iter.append(ite)
            ite += 1
            norm.append(norm_val)
            old_x = copy.deepcopy(x)
        new_x = copy.deepcopy(x)
        for i in range(n):
            new_x[i] = x[permute[i]]
        x = new_x
        return np.array(x),iter,norm
    
    def inbuilt(self,A_n,b):
        return np.linalg.solve(A_n,b)
    
    def get_roots(self,method = 'None',guess = np.array([])):
        match method:
            case 'gauss':
                return self.gauss(self.matrix,self.vector)
            case 'numpy':
                return self.inbuilt(self.matrix,self.vector)
            case 'seidal':
                return self.seidal(self.matrix,self.vector,guess)
            case 'jacobi':
                return self.jacobi(self.matrix,self.vector,guess)
            case 'None':
                return self.inbuilt(self.matrix,self.vector)

A = [[9,1,1],[2,10,3],[3,4,11]]
b = [10,19,0]
guess = [0,0,0]
ls = LinearSolve(A,b)
val_j,j_iter,j_norm = ls.get_roots(method = 'jacobi',guess = guess)
print('Roots are (jacobi method): ',val_j)
val_s,s_iter,s_norm = ls.get_roots(method = 'seidal',guess = guess)
print('Roots are (seidal method): ',val_s)
val_x = ls.get_roots(method='numpy')
print('Roots using numpy: ',val_x)
if val_j is not None:
    plt.plot(j_iter,j_norm,label = 'Jacobi Method')
    plt.plot(s_iter,s_norm,label = 'Seidal Method')
    plt.legend()
    plt.grid(True)
    plt.xlabel('Iterations')
    plt.ylabel('Distance between succesive values of x')
    plt.title('Convergence of Different Methods')
    plt.show()

A = [[2,10,3],[9,1,1],[3,4,11]]
b = [10,19,0]
guess = [0,0,0]
ls = LinearSolve(A,b)
val_j,j_iter,j_norm = ls.get_roots(method = 'jacobi',guess = guess)
print('Roots are (jacobi method): ',val_j)
val_s,s_iter,s_norm = ls.get_roots(method = 'seidal',guess = guess)
print('Roots are (seidal method): ',val_s)
val_x = ls.get_roots(method='numpy')
print('Roots using numpy: ',val_x)
if val_j is not None:
    plt.plot(j_iter,j_norm,label = 'Jacobi Method')
    plt.plot(s_iter,s_norm,label = 'Seidal Method')
    plt.legend()
    plt.grid(True)
    plt.xlabel('Iterations')
    plt.ylabel('Distance between succesive values of x')
    plt.title('Convergence of Different Methods')
    plt.show()

A = [[2,10,3],[9,9,1],[3,4,11]]
b = [10,19,0]
guess = [0,0,0]
ls = LinearSolve(A,b)
val_j,j_iter,j_norm = ls.get_roots(method = 'jacobi',guess = guess)
print('Roots are (jacobi method): ',val_j)
val_s,s_iter,s_norm = ls.get_roots(method = 'seidal',guess = guess)
print('Roots are (seidal method): ',val_s)
val_x = ls.get_roots(method='numpy')
print('Roots using numpy: ',val_x)
if val_j is not None:
    plt.plot(j_iter,j_norm,label = 'Jacobi Method')
    plt.plot(s_iter,s_norm,label = 'Seidal Method')
    plt.legend()
    plt.grid(True)
    plt.xlabel('Iterations')
    plt.ylabel('Distance between succesive values of x')
    plt.title('Convergence of Different Methods')
    plt.show()


# CODE - 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# %%
class Interpolation:
    def __init__(self,matrix,function = None) -> None:
        self.function = function
        self.matrix = matrix
        self.err = 1e-2
    def mypolyint(self,matrix,plot_poly = False):
        x = []
        y = []
        n = len(matrix)
        for i in range(n):
            x.append(matrix[i][0])
            y.append(matrix[i][1])
        poly_coef = np.zeros(n)
        for i in range(n):
            lan = np.poly1d([1])
            for j in range(n):
                if i != j:
                    lan *= (np.poly1d([1,-x[j]])/(x[i] - x[j]))
            poly_coef += y[i] * lan.coefficients
        polynomial = np.poly1d(poly_coef)
        if plot_poly:
            self.plot_fun(x,y,polynomial)
        return poly_coef,polynomial
    
    def divided_diff(self,x,y,dp,low,high):
        if low == high:
            return y[low]
        if dp[low][high] is not None:
            return dp[low][high]
        if high == low + 1:
            dp[low][high] = (y[high] - y[low])/(x[high] - x[low])
            return dp[low][high]
        dp[low][high] = (self.divided_diff(x,y,dp,low + 1,high) - self.divided_diff(x,y,dp,low,high - 1))/(x[high] - x[low])
        return dp[low][high]
    
    def mynewtonint(self,matrix,plot_poly = False):
        x = []
        y = []
        n = len(matrix)
        for i in range(n):
            x.append(matrix[i][0])
            y.append(matrix[i][1])
        poly_coef = np.zeros(n)
        polynomial = np.poly1d([y[0]])
        x_poly = np.poly1d([1])
        dp = [[None for _ in range(n)] for _ in range(n)]
        for i in range(1,n):
            x_poly *= np.poly1d([1,-x[i-1]])
            temp = polynomial + x_poly*self.divided_diff(x,y,dp,0,i)
            polynomial = temp
        poly_coef = polynomial.coefficients
        if plot_poly:
            self.plot_fun(x,y,polynomial)
        return poly_coef,polynomial
    
    def cubicSpline(self,matrix,plot_poly = False):
        x = []
        y = []
        n = len(matrix)
        for i in range(n):
            x.append(matrix[i][0])
            y.append(matrix[i][1])
        A = np.zeros((n-2,n-2))
        b = np.zeros(n-2)
        A[0][0] = (x[2] - x[0])/3
        A[0][1] = (x[2] - x[1])/6
        b[0] = ((y[2] - y[1])/(x[2] - x[1])) - ((y[1] - y[0])/(x[1] - x[0]))
        A[-1][-1] = (x[-1] - x[-3])/3
        A[-1][-2] = (x[-2] - x[-3])/6
        b[-1] = ((y[-1] - y[-2])/(x[-1] - x[-2])) - ((y[-2] - y[-3])/(x[-2] - x[-3]))

        for i in range(1,n-3):
            A[i][i-1] = (1/6)*(x[i+1] - x[i])
            A[i][i] = (1/3)*(x[i+2] - x[i])
            A[i][i+1] = (1/6)*(x[i+2] - x[i+1])
            b[i] = ((y[i+2] - y[i+1])/(x[i+2] - x[i+1])) - ((y[i+1] - y[i])/(x[i+1] - x[i]))
        #print(A)
        #print(b)
        m = np.dot(np.linalg.inv(A),b)
        #print(m)
        m = np.append(m,[0])
        m = np.insert(m,0,[0])
        C = np.zeros(n)
        D = np.zeros(n)
        poly_list = []
        poly_coef = []
        for i in range(1,n):
            D[i] = (y[i]/(x[i] - x[i-1])) - (m[i]/6)*(x[i] - x[i-1])
            C[i] = (y[i-1]/(x[i] - x[i-1])) - (m[i-1]/6)*(x[i] - x[i-1])
            polynomial = np.poly1d([1])
            polynomial = (np.poly1d([-1,x[i]])**3)*(m[i-1]/(6*(x[i] - x[i-1]))) \
                + (np.poly1d([1,-x[i-1]])**3)*(m[i]/(6*(x[i] - x[i-1]))) \
                + np.poly1d([-1,x[i]])*C[i] \
                + np.poly1d([1,-x[i-1]])*D[i]
            coef = polynomial.coefficients
            poly_coef.append(coef)
            poly_list.append(polynomial)
        if plot_poly:
            self.plot_cubic(x,y,poly_list)
        return poly_coef,poly_list
    def plot_cubic(self,x,y,poly_list):
        y_real = []
        x_r = []
        n = len(x)
        for i in range(1,n):
            x_range = np.linspace(x[i-1],x[i],1000)
            y_real.append(poly_list[i-1](x_range))
            x_r.append(x_range)
        x_r = np.array(x_r)
        y_real = np.array(y_real)
        x_r = x_r.reshape((1000*(n-1),))
        y_real = y_real.reshape((1000*(n-1),))
        plt.scatter(x,y,label = 'Points')
        plt.plot(x_r,y_real,label = 'Cubic Spline Interpolation')
        if self.function is not None:
            plt.plot(x_r,self.function(x_r),label = 'Original Function')
        plt.plot(x,y,label = 'Linear Piecewise Interpolation')
        plt.legend()
        plt.grid()
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Different Types of Interpolation')
        plt.show()
    
    def piecewise_linear_interpolation(self, plot_poly=False):
        x = [point[0] for point in self.matrix]
        y = [point[1] for point in self.matrix]
        
        poly_list_piece = []
        poly_coef_piece = []
        for i in range(1, len(x)):
            x0, x1 = x[i-1], x[i]
            y0, y1 = y[i-1], y[i]
            
            slope = (y1 - y0) / (x1 - x0)
            intercept = y0 - slope * x0
            
            polynomial = np.poly1d([slope, intercept])
            poly_list_piece.append(polynomial)
            poly_coef_piece.append([slope, intercept])

        return poly_coef_piece, poly_list_piece
    def plot_fun(self,x,y,polynom):
        plt.scatter(x,y,label = 'Points')
        n = len(x)
        x_range = np.arange(min(x)-0.1,max(x)+0.1,self.err)
        y_pred = polynom(x_range)
        plt.plot(x_range,y_pred,label = 'Interpolated Polynomial')
        if self.function is not None:
            plt.plot(x_range,self.function(x_range),label = 'Original Function')
        plt.legend()
        plt.grid(True)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.plot()        

# %% [markdown]
# # Q1)

# %%
matrix = [(1,1),(2,1/2),(3,1/3),(4,1/4)]
func = lambda x: 1/x
x = []
y = []
n = len(matrix)
for i in range(n):
    x.append(matrix[i][0])
    y.append(matrix[i][1])
ip = Interpolation(matrix,function=func)
coef, poly_list = ip.cubicSpline(matrix,plot_poly=True)
coef_linear,poly_linear = ip.piecewise_linear_interpolation(matrix)
print('Cubic Spline Coeffient : ',coef)
print('Piece-wise linear Coeffient : ',coef_linear)
plt.scatter(x,y)
for i in range(1,len(matrix)):
    x_r = np.linspace(matrix[i-1][0],matrix[i][0],1000)
    y_test = poly_list[i-1](x_r)
    plt.plot(x_r,y_test)
plt.grid()
plt.show()


# %%
matrix = [(-np.pi,0),(-np.pi/2,-1),(np.pi/2,1),(np.pi,0)]
func = lambda x: np.sin(x)
x = []
y = []
n = len(matrix)
for i in range(n):
    x.append(matrix[i][0])
    y.append(matrix[i][1])
ip = Interpolation(matrix,function=func)
coef, poly_list = ip.cubicSpline(matrix,plot_poly=True)
coef_linear,poly_linear = ip.piecewise_linear_interpolation(matrix)
print('Cubic Spline Coeffient : ',coef)
print('Piece-wise linear Coeffient : ',coef_linear)
plt.scatter(x,y)
for i in range(1,len(matrix)):
    x_r = np.linspace(matrix[i-1][0],matrix[i][0],1000)
    y_test = poly_list[i-1](x_r)
    plt.plot(x_r,y_test)
plt.grid()
plt.show()

# %% [markdown]
# # Q2)

# %%
matrix = [(0,2.5),(1,0.5),(2,0.5),(2.5,1.5),(3,1.5),(3.5,1.125),(4,0)]
x = []
y = []
n = len(matrix)
for i in range(n):
    x.append(matrix[i][0])
    y.append(matrix[i][1])
ip = Interpolation(matrix)
coef, poly_list = ip.cubicSpline(matrix,plot_poly=True)
coef_linear,poly_linear = ip.piecewise_linear_interpolation(matrix)
print('Cubic Spline Coeffient : ',coef)
print('Piece-wise linear Coeffient : ',coef_linear)
#print(coef)
plt.scatter(x,y)
for i in range(1,len(matrix)):
    x_r = np.linspace(matrix[i-1][0],matrix[i][0],1000)
    y_test = poly_list[i-1](x_r)
    plt.plot(x_r,y_test)
plt.grid()
plt.show()

# %% [markdown]
# # Q3)

# %% [markdown]
# 3a)

# %%
matrix = [(-0.5,0.731531),(0,1),(0.25,1.2684),(1,1.718282)]
func = lambda x: np.exp(x) - x**3
x = []
y = []
n = len(matrix)
for i in range(n):
    x.append(matrix[i][0])
    y.append(matrix[i][1])
ip = Interpolation(matrix,function=func)
coef, poly_list = ip.cubicSpline(matrix,plot_poly=True)
coef_linear,poly_linear = ip.piecewise_linear_interpolation(matrix)
print('Cubic Spline Coeffient : ',coef)
print('Piece-wise linear Coeffient : ',coef_linear)
#print(coef)
plt.scatter(x,y)
for i in range(1,len(matrix)):
    x_r = np.linspace(matrix[i-1][0],matrix[i][0],1000)
    y_test = poly_list[i-1](x_r)
    plt.plot(x_r,y_test)
plt.grid()
plt.show()

# %% [markdown]
# 3b)

# %%
err = []

for i in range(1,len(matrix)):
    x_r = np.linspace(matrix[i-1][0],matrix[i][0],1000)
    y_test = poly_list[i-1](x_r)
    err.append(abs(y_test - func(x_r)))
err = np.array(err)
err = err.reshape((1000*(n-1),))
x_r = np.linspace(matrix[0][0],matrix[-1][0],1000*(n-1))
print('Maximum error in cubic spline :',np.max(err))
plt.plot(x_r,err)
plt.grid()
plt.xlabel('x')
plt.ylabel('Error')
plt.title('x vs error curve in cubic spline')
plt.show()


# CODE - 3

In [ ]:
import pandas as pd

# %%
err = 1e-7

# %% [markdown]
# # Q1)
#  **$$f(x)=x^{6}-x-1$$**

# %%
def fun(x):
    return x**6 - x - 1

def dfun(x):
    return 6*(x**5) - 1

def newton_raphson(x0,err):
    data = []
    roots = []
    error = []
    present = x0
    fpresent = fun(x0)
    dfpresent = dfun(x0)
    next = present-(fpresent/dfpresent)
    roots.append(present)
    i = 1
    while(abs(next - present) > err):
        prev = present
        present = next
        fpresent = fun(present)
        dfpresent = dfun(present)
        next = present-(fpresent/dfpresent)
        data.append([i,present,fpresent,next,present - prev])
        roots.append(present)
        error.append(abs(present - prev))
        i += 1
    alpha = next
    data.append([i,next,fun(next),next - (fun(next)/dfun(next)),next - present])
    error.append(next- present)
    for i in range(0,len(data)):
        data[i].append(alpha - roots[i])
    
    df = pd.DataFrame(data,columns =['iter', 'x\u2099', 'f(x\u2099)', 'x\u2099\u208a\u2081', 'x\u2099 - x\u2099\u208b\u2081', 'a - x\u2099\u208b\u2081'])
    iter = np.arange(1,len(error)+1,1)
    plt.figure(1)
    plt.plot(iter,error,label = 'root = ' + str(roots[-1]))
    plt.legend()
    plt.xlabel('Iteration No.')
    plt.ylabel('Error')
    plt.title('Error vs Iteration Graph')
    plt.grid(True)
    plt.plot()
    plt.figure(2)
    r = [roots[-1]]*len(iter)
    plt.plot(iter,roots,label = '$x_n$')
    plt.plot(iter,r,label = 'root')
    plt.legend()
    plt.xlabel('Iteration No.')
    plt.ylabel('$x_n$')
    plt.title('Convergence of x towards the root')
    plt.grid(True)
    plt.plot()

    return df


# %%
df = newton_raphson(1.5,err)
df



# %%
df = newton_raphson(-1,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 1.5 the root is  1.134724145316218.
# 2. For the initial point -1 the root is -0.7780895986786547.

# %% [markdown]
# # Q2)

# %% [markdown]
# **$$f(x) = x^{3}-x^{2}-x-1$$**

# %%
def fun(x):
    return x**3 - x**2 - x - 1
def dfun(x):
    return 3*x**2 - 2*x - 1
df = newton_raphson(2,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 2,the root is 1.8392868100680193

# %% [markdown]
# **$$f(x) = 1 + 0.3*cos(x) $$**
# 

# %%
def fun(x):
    return 1 + 0.3*np.cos(x) - x
def dfun(x):
    return -1 - 0.3*np.sin(x)

df = newton_raphson(0,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 0,the root is 1.1284251543001005.

# %% [markdown]
# **$$f(x)= sin(x) + 1/2 - cos(x)$$**

# %%
def fun(x):
    return 0.5 + np.sin(x) - np.cos(x)

def dfun(x):
    return np.cos(x) + np.sin(x)
df = newton_raphson(0,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 0,the root is 0.42403103949074533

# %% [markdown]
# **$$ f(x) = e^{-x} - x$$**

# %%
def fun(x):
    return np.exp(-x) - x

def dfun(x):
    return -1 - np.exp(-x)
df = newton_raphson(1,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 1,the root is 0.537143285989123

# %% [markdown]
# **$$ f(x) = e^{-x} - sin(x)$$**

# %%
def fun(x):
    return np.exp(-x) - np.sin(x)
def dfun(x):
    return -np.exp(-x) - np.cos(x)
df = newton_raphson(1,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 1,the root is 0.5885327439585476

# %% [markdown]
# **$$ f(x) = x^3 - 2x - 2$$**

# %%
def fun(x):
    return x**3 - 2*x - 2
def dfun(x):
    return 3*x**3 - 2
df = newton_raphson(1,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 1,the root is 1.7692925029979065

# %% [markdown]
# **$$ f(x) = x^4 - x - 1$$**

# %%
def fun(x):
    return x**4 - x - 1
def dfun(x):
    return 4*x**3 - 1
df = newton_raphson(1,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 1,the root is 1.220744084605788

# %% [markdown]
# **$$ f(x) = tan(x) - x$$**

# %%
def fun(x):
    return np.tan(x) - x
def dfun(x):
    return (1/(np.cos(x)**2))- 1
df = newton_raphson(1,err)
df

# %% [markdown]
# Result:
# 1. For the initial point 1,the root is 2.7659680186998385e-07

# %% [markdown]
# # Q4)

# %% [markdown]
# **$$f(x) = a + x(x-1)^2 $$**

# %%
def newton_raphson(a,x0,err):
    roots = []
    present = x0
    fpresent = fun(a,x0)
    dfpresent = dfun(x0)
    next = present-(fpresent/dfpresent)
    roots.append(present)
    i = 1
    while(abs(next - present) > err):
        prev = present
        present = next
        fpresent = fun(a,present)
        dfpresent = dfun(present)
        next = present-(fpresent/dfpresent)
        roots.append(present)
        i += 1
    return roots

def fun(a,x):
    return a + x*((x-1)**2)
def dfun(x):
    return (x-1)**2 + 2*x*(x-1)

root0 = newton_raphson(0,-0.5,err)
root1 = newton_raphson(0,1.5,err)
r0 = [0]*len(root0)
r1 = [1]*len(root1)
iter0 = np.arange(1,len(root0)+1,1)
iter1 = np.arange(1,len(root1)+1,1)
plt.figure(1)
plt.plot(iter0,r0,label = 'Root = 0')
plt.plot(iter0,root0,label = 'Newton Raphson with Initial Guess = -0.5')
plt.legend()
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('Value of root')
plt.title('Convergence towards root')
plt.plot()

plt.figure(2)
plt.plot(iter1,r1,label = 'Root = 1')
plt.plot(iter1,root1,label = 'Newton Raphson with Initial Guess = 1.5')
plt.legend()
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('Value of root')
plt.title('Convergence towards root')
plt.plot()
a_val = [0,0.01,0.02,0.03,0.04,0.05,0.07,0.08,0.09,0.1]
roots0 = []
iter0 = []
roots1 = []
iter1 = []

for a in a_val:
    x = newton_raphson(a,0.99,err)
    roots0.append(x[-1])
    iter0.append(len(x))

for a in a_val:
    x = newton_raphson(a,1.01,err)
    roots1.append(x[-1])
    iter1.append(len(x))

plt.figure(3)
plt.plot(a_val,roots0,label = 'Initial Guess = ' + str(0.9),linewidth = 3)
plt.plot(a_val,roots1,label = 'Initial Guess = ' + str(1.1))
plt.legend()
plt.grid(True)
plt.xlabel('Value of a')
plt.ylabel('Negative Root Value')
plt.title('Convergence towards negative real root with increasing a')
plt.plot()

plt.figure(4)
plt.plot(a_val,iter0,label = 'Initial Guess = ' + str(0.9))
plt.plot(a_val,iter1,label = 'Initial Guess = ' + str(1.1))
plt.legend()
plt.grid(True)
plt.xlabel('a')
plt.ylabel('Iteration Count')
plt.title('Number of iteration in newton raphson method with increasing a')
plt.plot()

# %% [markdown]
# Result:
# 1. We notice, with increasing value of a , we can see the convergence towards the negative root of the function.
# 2. We also notice the relation between a and number of iterations to find the root are directly proportional.


# CODE - 4

In [ ]:


def leastSquareErrorPolynomial(matrix,m,plot_poly = False):
    if m >= len(matrix):
        m = len(matrix)-1
    x = []
    y = []
    n = len(matrix)
    for i in range(n):
        x.append(matrix[i][0])
        y.append(matrix[i][1])
    y = np.array(y)
    A = np.zeros((n,m+1))

    for i in range(n):
        for j in range(m+1):
            A[i][j] = x[i]**j
    A_T = np.transpose(A)
    ATA = np.dot(A_T,A)
    intermediate = np.dot(np.linalg.inv(ATA),A_T)
    coef = np.dot(intermediate,y)
    coef = coef[::-1]
    polynomial = np.poly1d(coef)
    if plot_poly:
        plot_fun(x,y,polynomial)
    return coef,polynomial
def plot_fun(x,y,polynom):
        err = 1e-3
        plt.scatter(x,y,label = 'Points')
        n = len(x)
        x_range = np.arange(min(x)-0.1,max(x)+0.1,err)
        y_pred = polynom(x_range)
        plt.plot(x_range,y_pred,label = 'Least Square Error Polynomial')
        # if self.function is not None:
        #     plt.plot(x_range,self.function(x_range),label = 'Original Function')
        plt.legend()
        plt.grid(True)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.plot()

x = [1,2,3,4,5]
y = [1,3,8,12,25]
mat = np.column_stack((x,y))
coef,poly_i = leastSquareErrorPolynomial(mat,m=3,plot_poly=True)
print('Coefficient of Least Square Error Polynomial : ',coef)

poly_lst = []
poly_coef = []
x_r = np.linspace(min(x)-0.1,max(x)+0.1,1000)
for i in range(1,len(x)):
    coef,poly_i = leastSquareErrorPolynomial(mat,i)
    poly_coef.append(coef)
    poly_lst.append(poly_i)
    plt.plot(x_r,poly_i(x_r),label = str(i) + ' Order')
plt.scatter(x,y,label = 'Points')
plt.legend()
plt.grid(True)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Least Square Error Polynomial with different Orders')
plt.plot()
